In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
import numpy as np
import os 
import time
!python -m spacy download en_core_web_lg
import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [22]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
# Read, then decode for py2 compat.
path_to_file = '/content/drive/MyDrive/466 data/en_tech_train.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

path_to_test_file = '/content/drive/MyDrive/466 data/en_tech_test.txt'
test_text = open(path_to_test_file, 'rb').read().decode(encoding='utf-8')

path_to_validation_file = '/content/drive/MyDrive/466 data/en_tech_validate.txt'
validation_text = open(path_to_validation_file, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

In [ ]:
processed_test_set = test_text.split("\n")
processed_validate_set = validation_text.split("\n")

input_size = 21
test_sentences = []
validate_sentences = []

#train_sentences[0:10]
for line in processed_test_set:
    temp = line.split()
    for i in range(input_size, len(temp)):
        seq = temp[i - input_size:i]
        temp2 = " ".join(seq)
        test_sentences.append(temp2)
print(test_sentences[0:10])
for line in processed_validate_set:
    temp = line.split()
    for i in range(input_size, len(temp)):
        seq = temp[i - input_size:i]
        temp2 = " ".join(seq)
        validate_sentences.append(temp2)

In [31]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

In [33]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

In [35]:
seq_length = 20
examples_per_epoch = len(text)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

In [38]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

In [40]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

In [ ]:
# Batch size
BATCH_SIZE = 400

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

In [43]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [44]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [45]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [46]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(400, 20, 42) # (batch_size, sequence_length, vocab_size)


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
model.summary()

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

In [ ]:
tf.exp(mean_loss).numpy()

In [50]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [53]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [55]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['', '[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [56]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [57]:
def predict(one_step_model, next_char, states=None):
  next_char = tf.constant([next_char])
  result = [next_char]
  states = None
  while True:
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    if next_char == " " or next_char == "\r" or next_char == "\n":
      break
    result.append(next_char)
  
  result = tf.strings.join(result)
  result = (result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)[0]
  result = result.strip()
  result = result.split()
  result = result[-1]
  return result

In [58]:
def validate(sentences, nlp):
  new = []
  for i in range(len(sentences)):
    new.append(sentences[i].split())
  validate_sequences = np.array(new)
  validate_X, validate_y = validate_sequences[:, :-1], validate_sequences[:, -1]
  print(len(validate_X))
  average_accuracy = 0
  num = 0
  for i in range(len(validate_X)):
    prediction = predict(one_step_model, ' '.join(validate_X[i])+' ')
    x = nlp(prediction)
    y = nlp(str(validate_y[i]))

    if (x[0].has_vector) and (y[0].has_vector):
      average_accuracy += nlp(prediction).similarity(nlp(str(validate_y[i])))
      num += 1
  print("num is", num)
  average_accuracy /= num
  return average_accuracy

In [ ]:
accuracy = validate(test_sentences, nlp)

In [ ]:
accuracy